# ------ **Import Subroutines and Settings** -------

# *Procedure* for building " *The df_ib DataFrame from ib770 & ib970 combined with meta_data*

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [4]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Pre-concatinate processing import and prepare 
1. **"df_m_ib_tst"**
2.**"df_ib77_tst"**


## def functions called in data importing & refinment.

In [5]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading or trailing numbers (and optional separators)
    from column names without altering descriptor text.
    Examples cleaned:
      '1 Weight' → 'Weight'
      'Weight 1' → 'Weight'
      '2-Height' → 'Height'
      '3_TBW' → 'TBW'
      '4:LBM' → 'LBM'
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_:]*', '', col)
        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_:]*\d+$', '', cleaned)
        new_cols[col] = cleaned

    df = df.rename(columns=new_cols)
    return df


In [6]:
#filling the media cols
def fill_ib_leading_columns(df):
    """
    Fills the 5 leading operator columns for InBody datasets:
      - timestamp  ← parsed from 'Test Date / Time' (YYYYMMDDHHMMSS)
      - dtv        ← days since 1900‑01‑01
      - ib_id      ← 'mrn' if test time 03:00–23:59, else 'eve'
      - cls        ← NaN
      - cmmnts     ← NaN
    """

    df = df.copy()

    # --- 1. timestamp (correct parsing) ------------------------
    df['timestamp'] = pd.to_datetime(
        df['Test Date / Time'].astype(str),
        format="%Y%m%d%H%M%S",
        errors="coerce"
    )

    # --- 2. dtv: days since 1900‑01‑01 -------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id classification -------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4. cls ------------------------------------------------
    df['cls'] = np.nan

    # --- 5. cmmnts ---------------------------------------------
    df['cmmnts'] = np.nan

    return df


In [7]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [8]:
# remove duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


# UPDATE **"df_m_ib_tst"** with latest data

## Bringing in **df_m_ib_tst_nn**  the main data file from Excel

In [11]:
#this brings in the main data file from Excel
df_m_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv")
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
# verify 
df_m_ib_tst_nn
print("imported **df_m_ib_tst_nn** ie numbers removed from col heads")

imported **df_m_ib_tst_nn** ie numbers removed from col heads


In [17]:
# df_m_ib_tst_nn slicing
# verify 
print('df_m_ib_tst_nn imported \n Test Date / Time \n',df_m_ib_tst_nn['Test Date / Time'])  # list form wo meta


df_m_ib_tst_nn imported 
 Test Date / Time 
 0     20251213054921
1     20251214234814
2     20251217075000
3     20251218081604
4     20251216233809
5     20251215080038
6     20251212221713
7     20251219080729
8     20251214090336
9     20251218212747
10    20251219232758
11    20251215233324
12    20251216064808
13    20251211092610
14    20251217230745
15    20251220085024
16    20251212084231
17    20251213054921
18    20251214234814
19    20251217075000
20    20251218081604
21    20251216233809
22    20251215080038
23    20251212221713
24    20251219080729
25    20251214090336
26    20251218212747
27    20251219232758
28    20251215233324
29    20251216064808
30    20251211092610
31    20251217230745
32    20251220085024
33    20251212084231
34    20251213054921
35    20251214234814
36    20251217075000
37    20251218081604
38    20251216233809
39    20251215080038
40    20251212221713
41    20251219080729
42    20251214090336
43    20251218212747
44    20251219232758
45    2025

# WORKING cell

# ADD MEDA , CLEAN AND SORT  >>>>   **df_m_ib_tst_nn**

## Fill in the meta columns by reading the data in the rest of the columns >>> **df_m_ib_tst_nn**

In [ ]:
# fill in meta columns
df_m_ib_tst_nn = fill_ib_leading_columns(df_m_ib_tst_nn)
print("meta cols filled")
# verify df_m_ib_tst_nn

#### Test the **"media_lst"** calculations of the timestamp, DTV, ib_id , cls , cmmnts

In [ ]:
media_lst = [
    "Test Date / Time",
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]


In [ ]:
# verify df_m_ib_tst_nn[media_lst].head(20)

## Sort the data frame on the basis of timestamp you >> **df_m_ib_tst_s**

In [ ]:
df_m_ib_tst_nn_s = sort_by_timestamp(df_m_ib_tst_nn)

In [ ]:
# verify 
df_m_ib_tst_nn_s[media_lst].head(20)

In [ ]:
#verify df_m_ib_tst_nn_s[["dtv", "ib_id" , "SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

## Remove duplicates [...xd]  Retaining the first of a duplicate pair  >>> **df_m_ib_tst_s_xd**

In [ ]:
df_m_ib_tst_nn_s_xd  =  remove_ib_duplicates(df_m_ib_tst_s, subset_cols=None)
# verify df_m_ib_tst_nn_s_xd

## Slicing the fully processed data frame [df_m_ib_tst_nn_s_xd] to display columns of interest  

In [ ]:
#verify 
df_m_ib_tst_nn_s_xd[["dtv","timestamp","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

# The stored data is loaded checked and verified, and ready to receive new data
1. **"df_m_ib_tst"**
2. **"df_m_ib_tst_nms"**
3. **"df_ib97_tst"**
4. **"df_ib97_tst_nms"**
5. **"df_ib77_tst"**
6. **"df_ib77_tst_nms"**
7. **"_______________"**

In [ ]:
df_m_ib_tst_nms = list(df_m_ib_tst.columns)
#df_ib77_raw_nms with numbers 
#verify  df_m_ib_tst_nms
print("df_m_ib_tst_nms w/o numbers OK")

In [ ]:
# Testing duplicates in the column titles
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print("df_m_ib_tst_nms w/o numbers no duplicate OK")

## Read the latest data from 770 and 970 in the data folder of repo

### ib77:  This segment reads the data from the Excel file_ib77 and computes the names of the column heads, strips them of numbers, puts them in a List, and records them in a pickle. *ib77_raw_nms*

In [ ]:
# loads the new from the 770
df_ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# verify 
df_ib77_raw
#print("df_ib77_raw w/o numbers OK")

In [ ]:
df_ib77_nn = strip_numbers_from_columns(df_ib77_raw)
#verify 
print(list(df_ib77_nn.columns))

df_ib77_nn[['. ID','. Test Date / Time','. ECW/TBW']]  #,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [ ]:
df_ib77_raw_nms = list(df_ib77_raw.columns)
# VERIFY df_ib77_raw_nms
print ("df_ib77_raw_nms with numbers OK")

In [ ]:
# Testing duplicates in the column titles
df_ib77_raw.columns[df_ib77_raw.columns.duplicated()]
print ("df_ib77_raw_nms with numbers no duplicates OK")

In [ ]:
# strip numbers off So that the identical names in ib77 are the same as 97
df_ib77_raw_nms_nn = strip_col_numbers(df_ib77_raw)
# Verify 
df_ib77_raw_nms_nn 
#print ("df_ib77_raw_nms_nn ie. w/o numbers  OK")

In [ ]:
# Testing duplicates in the column titles
df_ib77_raw.columns[df_ib77_raw.columns.duplicated()]
print ("w/o numbers no duplicates OK")

In [ ]:
# Load the IB77 Columns without numbers to pickle 
with open("df_ib77_raw_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_nms, f)
print ("df_ib77_raw_nms.pkl OK")

In [ ]:
# READ the IB77 Columns without numbers FROM pickle 
with open("df_ib77_raw_nms.pkl", "rb") as f:
    df_ib77_raw_nms = pickle.load(f)
# Verify df_ib77_raw_nms
print ("from pkl df_ib77_raw_nms.pkl OK")

#### Calculate excess columns in the main data frame compared the columns in the IB77 data frame.

In [ ]:
# Verify that we have the main data frame
df_m_ib_tst.columns

In [ ]:
#Verify that we have data from data frame to be added to the main data frame If it's not there read it from pickle
# verify df_ib77_raw.columns

# This will load the ib77 into the accumulated data

In [ ]:
# verify df_m_ib_tst
print(" The main df_m_ib_tst is in place with col numbers:")

In [ ]:
# verify df_m_ib_tst_nn
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
df_m_ib_tst_nn
print(" The main df_m_ib_tst_nn is in place w/o col numbers:")

In [ ]:
# This is the IB77 columns that still have the numbers attached
ib77_raw_missing_cols_n = [col for col in ib77_raw.columns 
                           if col not in df_m_ib_tst_nn.columns]

# verify ib77_raw_missing_cols

In [ ]:
for col in ib77_raw_missing_cols:
    ib77_raw[col] = np.nan
ib77_raw              # ["ECW/TBW"]

In [ ]:
import pandas as pd

# Suppose missing_cols is a list of column names you want to add
new_cols = pd.DataFrame({col: [None] * len(ib77_raw) for col in missing_cols})
new_cols

In [ ]:
# Add them in one operation
ib77_raw = pd.concat([ib77_raw, new_cols], axis=1)

In [ ]:
# Add any missing columns to IB77_raw
for col in df_m_ib_tst.columns:
    if col not in ib77_raw.columns:
        ib77_raw[col] = None
# verify 
ib77_raw

In [ ]:
df_ib77_raw[["BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [ ]:
# The main data frame is larger than the add data frame so we need To determine the Missing columns.
missing_cols = [c for c in df_m_ib_tst.columns if c not in df_ib77_raw.columns]
#verify missing_cols

In [ ]:
# Use the missing_cols dataframe in order add columns with NAn values to the ib77 values so it can be appended to with the main data frame "df_m_ib_tst
df_ib77_cncnt = df_ib77_raw.copy()    # start building the df
#verify df_ib77_cncnt

In [ ]:
# 1. Add missing columns
for col in missing_cols:
    df_ib77_cncnt[col] = np.nan
#verify df_ib77_cncnt

In [ ]:
# 3. Reorder to match master schema
df_ib77_cncnt = df_ib77_cncnt[df_m_ib_tst.columns]
#verify df_ib77_cncnt

In [ ]:
# 4. Append
df_m_ib_tst = pd.concat([df_m_ib_tst, df_ib77_cncnt], ignore_index=True)
#verify df_m_ib_tst

In [ ]:
# Calculate missing columns when df_ib77 raw data is appended to IB_tst Data frame
df_ib77_raw_missing_cols_n = [col for col in df_ib77_raw.columns if col not in df_m_ib_tst.columns]
# verify 
df_ib77_raw_missing_cols_n
print ("df_ib77_raw_missing_cols_n with numbers  OK")

In [ ]:
# strip numbers off So that the identical names in ib77 are the same as 97
df_ib77_raw_missing_cols_nms = strip_col_numbers(df_ib77_raw_missing_cols_n)
# Verify 
df_ib77_raw_missing_cols_nms
print ("df_ib77_raw_missing_cols_nms w/o numbers  OK")

In [ ]:
# Load the df_ib77_raw_missing_cols_nms to pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_missing_cols_nms, f)
print ("df_ib77_raw_missing_cols_nms.pkl OK")

In [ ]:
# READ thedf_ib77_raw_missing_cols_nms FROM pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "rb") as f:
    df_ib77_raw_missing_cols_nms = pickle.load(f)
# Verify df_ib77_raw_missing_cols_nms
print ("from pkl df_ib77_raw_missing_cols_nms.pkl OK")

In [ ]:
# Calculate missing columns when df_ib77 raw data is appended to ib_tst DataFrame

## Calculate the **ib_tst** columns that are not used when **ib77** is concatenated =  **xcss77**

### ib97:  This segment Reads the data from the excel file_ib97 and computes names of the column head, strips them of numbers and Puts them in a list and records them in a pickle. *ib97_raw_nms*

In [ ]:
# loads the new from the 970
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97", 
    pattern="*"
)

In [ ]:
ib97_raw_nms = list(ib97_raw.columns)
# Verify 
ib97_raw_nms

In [ ]:
# strip numbers off So that the identical names in ib97 are the same as ib97
ib97_raw_nms = strip_col_numbers(ib97_raw)
# Verify
ib97_raw_nms

In [ ]:
# Load the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "wb") as f:
    pickle.dump(ib97_raw_nms, f)


In [ ]:
# READ the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "rb") as f:
    ib97_raw_nms = pickle.load(f)
# Verify ib97_raw_nms

#### The following data are loaded and ready to be consolidated into an updated Total data data frame "m_bi_tst" from :XL >>> "df_m_bi_tst" and new"df_ib97_raw",new "df_ib77_raw"

### This will load the ib97 into the accumulated data

In [ ]:
# Add any missing columns to IB97_raw
for col in m_ib_tst.columns:
    if col not in ib97_raw.columns:
        ib97_raw[col] = None
# Reorder IB97_raw columns to match m_ib_tst
ib97_raw = ib97_raw[m_ib_tst.columns]


In [ ]:
m_ib_tst = pd.concat([m_ib_tst, ib97_raw], ignore_index=True)
m_ib_tst

In [ ]:
# Reorder ib77_raw columns to match df_m_ib_tst
ib77_raw = ib77_raw[df_m_ib_tst.columns]

In [ ]:
# This concatenates 770 new data with the existing data That contains the accumulator data from 770 and 970
df_m_ib_tst = pd.concat([df_m_ib_tst, ib77_raw], ignore_index=True)
#verify df_m_ib_tst

In [ ]:
# This checks to see if column names are still all unique after the new data
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print(" No duplicates found Afternoon data was appended")

In [ ]:
df_m_ib_tst["

In [ ]:
ib77_raw.columns[ib77_raw.columns.duplicated()]


In [ ]:
df_m_ib_tst_s_xd[["dtv","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

# load latest [m_ib_tst] to xl

In [ ]:
# load latest [m_ib_tst] from_xl
folder_path = "/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv"
m_ib_tst.to_csv(folder_path, index=False)
# verify
#
m_ib_tst

# Everything below this line is residual the days the template was developed and completed. Erase it when everything using the template is completed
# ================================================================================================================================================================

# Startup only Create  **df_ib_fls_tmplt**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

### Both the 77 and the 90 test datasets are convrted to dfs In order to determine *df_ib_tst_nms* Which is a single column data frame that all of the column names of the *df_ib_tst*

#### ✅ Concatenate → Preserve Order → Drop Duplicates

In [ ]:
####  Concatenate → Preserve Order → Drop Duplicates mmmmmmmmmnnnnnnnllllll
# Concatenate in the required order
df_ib_tst_nms = pd.concat(
    [df_ib77_raw_nms, df_ib97_raw_nms],
    axis=0,
    ignore_index=True
)
# print(df_ib_tst_nms)
# Remove duplicate rows, keeping the first occurrence (from df_ib77_raw_nms)
df_ib_tst_nms = df_ib_tst_nms.drop_duplicates(keep="first")
# verify 
# print(df_ib_tst_nms)

In [ ]:
# Verifies that the duplicates have been removed and that the list has been compacted and there are not duplicates of 770 and 970 Col
df_ib_tst_nms[df_ib_tst_nms.duplicated()]

### *df_ib_tst_nms* must be converted to a simple list in order to made equal to the column heads of df_tst

In [ ]:
# The large test data frame that will be used to accept values of each from both 770 N 970 will be built on the basis of the df_ib_tst_nms, but this data bugs be in form of a strings.
# df['colname'].astype(str).tolist       # sign This is the sample given from copilot that is modified below for my situation 

ib_tst_lst = df_ib_tst_nms[0].astype(str).tolist()
# verify 
# print(ib_tst_lst)

In [ ]:
# this makes the template for the ib_tst data For both the 770 and the 970 It is made up of a 770 and a 970 with duplicates eliminated in a list of strings

df_ib_tst = pd.DataFrame(
    data = [[""] * len(df_ib_tst_lst)] * 2,
    columns = ib_tst_lst
)
#df_ib_tst_nms
df_ib_tst

In [ ]:
# Tests 
df_ib_tst["ID"]


In [ ]:
df_ib_tst.columns


In [ ]:
df_ib_tst["ID"]

## data frame of the main data storage_“ib_tst"

###  Make the edits to the structure of the main data spreadsheet that represents structure

### Then the CSV file into the worksheet to display the changes in Jupiter lab

In [ ]:
df_loaded = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/ib_tst.csv")
df_loaded.columns

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib_tst/ib_tst.csv"
df_ib_tst.to_csv(folder_path, index=False)
df_ib_tst

In [ ]:
df_loaded["Age"]



In [ ]:
#==================================================================================================================== Oh

###


# Align the columns via index suggestions from Copilot to insert new data into the major data frame

Put the creation of a data frame into its own worksheet

# ==============================================================================================================================================

In [ ]:
# def strip_numbers_from_columns(df):
import re
def strip_numbers_from_columns(df):
    """
    Removes leading or trailing numbers (and optional separators)
    from column names without altering descriptor text.
    Examples cleaned:
      '1 Weight' → 'Weight'
      'Weight 1' → 'Weight'
      '2-Height' → 'Height'
      '3_TBW' → 'TBW'
      '4:LBM' → 'LBM'
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_:]*', '', col)
        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_:]*\d+$', '', cleaned)
        new_cols[col] = cleaned

    df = df.rename(columns=new_cols)
    return df
